In [9]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from netgen.csg import Pnt


In [10]:
#help(unit_square)
l = 2

mw = 1
for i in range(l):
    mw = mw/2

#netgen_mesh = unit_square.GenerateMesh(maxh=mw, segmentsperedge=l+1.2, grading=0.1)

netgen_mesh = unit_square.GenerateMesh(maxh=1)
mesh=Mesh(netgen_mesh)

for s in range(l):
    mesh.Refine()

# # barycentric refinement crashes programm if mesh hierarchy isnt deleted, 
# # which can be done by saiving and loading the mesh
mesh.ngmesh.Save("mesh")
del mesh
mesh = Mesh("mesh.vol.gz")

mesh._updateBuffers()

#mesh = Mesh(netgen_mesh)
#help(netgen_mesh)
unref_verts = mesh.vertices

unref_edges = mesh.edges

nedges = mesh.nedge

parent_el = mesh.ne

#H_h = HDiv(mesh,order=0,RT=True) 
H_h = HCurl(mesh,order=1,type1=True)

for e in mesh.edges:
    print(e.nr, H_h.GetDofNrs(e))

unref_dofnr = H_h.ndof #mesh.ne
print(unref_dofnr)
print(mw)
elvol = Integrate(CoefficientFunction(1),mesh,element_wise=True)
mesh_h = [(2*vol)**(1/2) for vol in elvol]
print("actual meshwidth range",min(mesh_h),max(mesh_h), "\n")

#Draw(mesh)

#help(la.SparseMatrixdouble)

0 (0,)
1 (1,)
2 (2,)
3 (3,)
4 (4,)
5 (5,)
6 (6,)
7 (7,)
8 (8,)
9 (9,)
10 (10,)
11 (11,)
12 (12,)
13 (13,)
14 (14,)
15 (15,)
16 (16,)
17 (17,)
18 (18,)
19 (19,)
20 (20,)
21 (21,)
22 (22,)
23 (23,)
24 (24,)
25 (25,)
26 (26,)
27 (27,)
28 (28,)
29 (29,)
30 (30,)
31 (31,)
32 (32,)
33 (33,)
34 (34,)
35 (35,)
36 (36,)
37 (37,)
38 (38,)
39 (39,)
40 (40,)
41 (41,)
42 (42,)
43 (43,)
44 (44,)
45 (45,)
46 (46,)
47 (47,)
48 (48,)
49 (49,)
50 (50,)
51 (51,)
52 (52,)
53 (53,)
54 (54,)
55 (55,)
56
0.25
actual meshwidth range 0.2499999999999995 0.2499999999999996 



In [65]:
gfu = GridFunction(H_h, multidim = H_h.ndof)

for i in range(H_h.ndof):
    gfu.vecs[i][i] = 1.0


Draw(gfu, animate = False, min = 0, max = 1, deformation = False)
help(H_h)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Help on HCurl in module ngsolve.comp object:

class HCurl(FESpace)
 |  Keyword arguments can be:
 |  
 |  order: int = 1
 |    order of finite element space
 |  complex: bool = False
 |    Set if FESpace should be complex
 |  dirichlet: regexpr
 |    Regular expression string defining the dirichlet boundary.
 |    More than one boundary can be combined by the | operator,
 |    i.e.: dirichlet = 'top|right'
 |  dirichlet_bbnd: regexpr
 |    Regular expression string defining the dirichlet bboundary,
 |    i.e. points in 2D and edges in 3D.
 |    More than one boundary can be combined by the | operator,
 |    i.e.: dirichlet_bbnd = 'top|right'
 |  dirichlet_bbbnd: regexpr
 |    Regular expression string defining the dirichlet bbboundary,
 |    i.e. points in 3D.
 |    More than one boundary can be combined by the | operator,
 |    i.e.: dirichlet_bbbnd = 'top|right'
 |  definedon: Region or regexpr
 |    FESpace is only defined on specific Region, created with mesh.Materials('regexpr')
 

In [66]:
#Important

elnrs = np.ones((nedges,2),dtype=int)

for e in mesh.edges:
    el1 = mesh.__getitem__(e.elements[0])
    elnrs[e.nr,:] = el1.nr
    if len(e.elements) == 2:
        el2 = mesh.__getitem__(e.elements[1])
        elnrs[e.nr,1] = el2.nr

#print(elnrs)

In [67]:


mesh.ngmesh.SplitPowellSabin()
mesh.ngmesh.Update()

mesh._updateBuffers()

#H_h.Update() 

H_h = Compress(HCurl(mesh,order=1,type1=True))


#might need to be number of faces in barycentrically refined mesh
bary_dofnr = H_h.ndof #mesh.ne
print(bary_dofnr)

#mesh._updateBuffers()


#Draw(mesh)

322


In [68]:
data_length = 24 * nedges

data = np.zeros(data_length)
row_ind = np.zeros(data_length, dtype=int)
col_ind = np.zeros(data_length, dtype=int)

j = 0

for e in unref_edges:
    parent = e.nr  
    #print(parent)
    v0 = e.vertices[0]  
    v1 = e.vertices[1] 
    v0 = mesh.__getitem__(v0)
    v1 = mesh.__getitem__(v1)
    verts = [v0,v1]
    coeffs = [-1,1]

    i = 1
    for vi in verts:
        for el in vi.elements:
            if mesh.ngmesh.MacroElementNr(el.nr) == elnrs[parent,0] or mesh.ngmesh.MacroElementNr(el.nr) == elnrs[parent,1]:
                el = mesh.__getitem__(el)
                for child in el.edges:
                    
                    child = mesh.__getitem__(child)
                    if len(child.elements) == 2:
                        
                        par_el1 = mesh.ngmesh.MacroElementNr(child.elements[0].nr)
                        par_el2 = mesh.ngmesh.MacroElementNr(child.elements[1].nr)

                        #same parent element
                        if  par_el1 == par_el2:
                            row_ind[j] = parent
                            col_ind[j] = H_h.GetDofNrs(child)[0]
                            data[j] = coeffs[i] * 1/6 
                            j += 1
                        #different parent, but both are elements of parent edge (the children of the original edge fulfill this)
                        elif par_el1 != par_el2 and (par_el1 == elnrs[parent,0] or par_el1 == elnrs[parent,1]) and (par_el2 == elnrs[parent,0] or par_el2 == elnrs[parent,1]):
                            row_ind[j] = parent
                            col_ind[j] = H_h.GetDofNrs(child)[0]
                            data[j] = coeffs[i] * 3/12 
                            j += 1
                    #boundry
                    elif len(child.elements) == 1:
                        #the edge midpoint always has higher number than the coarse vertices, s it will always be at index 1
                        v2 = child.vertices[1]
                        v2 = mesh.__getitem__(v2)
                        #the edges which correspond to the original edge in case it was on a boundry,
                        #check if this vertex shares an edge with the other vertex of the original edge
                        #this excludes boundry edges that arent part of the original edge
                        otherv = verts[i]
                        if(set(v2.edges).intersection(otherv.edges) != set()):
                            row_ind[j] = parent
                            col_ind[j] = H_h.GetDofNrs(child)[0]
                            data[j] = coeffs[i] * 3/6 
                            j += 1 

        i -= 1                   

ng_mat = la.SparseMatrixdouble.CreateFromCOO(row_ind,col_ind,data, nedges,mesh.nedge - nedges)
#print(ng_mat.ToDense())

In [69]:
# dfu = GridFunction(H_h,multidim = unref_dofnr)
# #help(ng_mat)



# dfu.Set((0,0))
# for i in range(unref_dofnr):
#     #print(ng_mat.ToDense().NumPy()[:,i])
#     dfu.vecs[:][i] = ng_mat.ToDense().NumPy()[i,:]


# # for i in range(4):
#     #print(dfu.vecs[i][:].FV().NumPy())
# Draw(dfu, animate = False, min = 0, max = 2, deformation = False)

In [70]:
# #TO compare with

# netgen_mesh = unit_square.GenerateMesh(maxh=1) #(maxh=mw, segmentsperedge=l+1.2, grading=0.1)

# ref_mesh = Mesh(netgen_mesh)

# for s in range(l):
#     ref_mesh.Refine()

# H_h = Compress(HDiv(ref_mesh, order=0, RT=True)) #HCurl(ref_mesh,order=1,type1=True))

# gfu = GridFunction(H_h,multidim = H_h.ndof)

# #gfu.Set(0) 

# for i in range(H_h.ndof):
#     gfu.vecs[i][i] = 1.0

# Draw(gfu, animate = False, min = 0, max = 2.0, deformation = False)

In [71]:
#DUAl, comment out reference visual above

# TODO direction of vectors is inconsistenf for edges that contain 2 boundary vertices at the top

dual_space = Compress(HDiv(mesh,order=0,RT=True)) # HCurl(mesh, order=1, type1=True)) #


In [72]:
def edge_length(e, mesh):
    p1 = mesh.__getitem__(e.vertices[1]).point
    p0 = mesh.__getitem__(e.vertices[0]).point
    e_len = sqrt((p1[0] - p0[0])**2 + (p1[1] - p0[1])**2)
    return e_len

def append_next_edge(edges, prev_edge, mesh, vi, v2):
    for el in prev_edge.elements:
        el = mesh.__getitem__(el)
        for next_e in el.edges:
            next_e = mesh.__getitem__(next_e)
            if (next_e not in edges) and (vi in next_e.vertices) and (v2 not in next_e.vertices):
                edges.append(next_e)
                return
            
def get_ref_edge(central_edge, mesh, vi, v2):
    for el in central_edge.elements:
        el = mesh.__getitem__(el)
        for next_e in el.edges:
            next_e = mesh.__getitem__(next_e)
            if (vi in next_e.vertices) and (v2 in next_e.vertices):
                return next_e

In [73]:
data_length = 0
for e in unref_edges:
    for v in e.vertices:
        v = mesh.__getitem__(v)
        data_length += len(v.edges) + 4

data = np.zeros(data_length)
row_ind = np.zeros(data_length, dtype=int)
col_ind = np.zeros(data_length, dtype=int)

j = 0

for e in unref_edges:
    #getting accesible information
    parent = e.nr  
    v0 = e.vertices[0]  
    v1 = e.vertices[1] 
    v0 = mesh.__getitem__(v0)
    v1 = mesh.__getitem__(v1)

    bnd_edge = False
    if elnrs[parent, 0] == elnrs[parent,1]:
        bnd_edge = True

    #finding the central dual edges of the reference edge
    central_edges = []

    for el0 in v0.elements:
        for el1 in v1.elements:
            el0 = mesh.__getitem__(el0)
            el1 = mesh.__getitem__(el1)
            intersect = tuple(set(el0.edges).intersection(el1.edges))
            if intersect != ():
                e = mesh.__getitem__(intersect[0])
                central_edges.append(e)

    #finding vertex in edge midpoint
    if len(central_edges) == 2:
        v2 = tuple(set(central_edges[0].vertices).intersection(central_edges[1].vertices))[0]
        v2 = mesh.__getitem__(v2)
    else:
        for v in central_edges[0].vertices:
            v = mesh.__getitem__(v)
            if len(v.faces) == 2:
                v2 = v

    # #############################################################
    # if len(central_edges) == 2:
    #     v3 = mesh.__getitem__(central_edges[0].vertices[1])
    #     assert(v3 != v2 and v3.nr > v2.nr)
    #     vecA = [v1.point[0] - v0.point[0], v1.point[1] - v0.point[1]]
    #     vecB = [v3.point[0] - v2.point[0], v3.point[1] - v2.point[1]]

    #     if vecA[0] * vecB[1] - vecA[1] * vecB[0] > 0:
    #         tmp = central_edges[0]
    #         central_edges[0] = central_edges[1]
    #         central_edges[1] = tmp
    #         print(central_edges)
    # ###############################################################

    #setting the coefficients for the edges
    verts = [v0,v1]
    sign = [1, -1]
    if bnd_edge == False:
        for k in range(2):
            vi = verts[k]
            Nc = len(vi.faces)/2
            Ne = len(vi.edges)

            bnd_vertex = False
            for bnd_el in mesh.Elements(BND):
                if vi in bnd_el.vertices:
                    bnd_vertex = True

            s = sign[k]
            edges = []

            if bnd_vertex == False:
                #INTERNAL vertex ---------------------------------------------------------------------
                edges = [central_edges[k]]
                
                i = 0    
                for e_i in edges:            
                    
                    #calculate edge length
                    e_len = edge_length(e_i, mesh)
                    coeff = (Nc - i)/(2*e_len*Nc) 
                    row_ind[j] = parent
                    col_ind[j] = dual_space.GetDofNrs(e_i)[0]
                    data[j] = coeff *s
                    j += 1

                    append_next_edge(edges, e_i, mesh, vi, v2)
                    i += 1
            else:
                #BND vertex --------------------------------------------------------------------------
                #deal with reference edge (part of original edge)
                ref_edge = get_ref_edge(central_edges[0], mesh, vi, v2)
                e_len = edge_length(ref_edge, mesh)
                coeff = (2 - Nc)/(2*e_len*Nc) 
                row_ind[j] = parent
                col_ind[j] = dual_space.GetDofNrs(ref_edge)[0]
                data[j] = coeff *s
                j += 1

                #deal with central edge
                e_len = edge_length(central_edges[k], mesh)
                coeff = (1)/(2*e_len) 
                row_ind[j] = parent
                col_ind[j] = dual_space.GetDofNrs(central_edges[k])[0]
                data[j] = coeff *s
                j += 1

                #edges with counterclockwise lower index than ref edge
                append_next_edge(edges, central_edges[k-1], mesh, vi, v2)
                for e_i in edges:            
                    #calculate edge length
                    e_len = edge_length(e_i, mesh)
                    coeff = (1-Nc)/(e_len*Nc) 
                    row_ind[j] = parent
                    col_ind[j] = dual_space.GetDofNrs(e_i)[0]
                    data[j] = coeff * s 
                    j += 1

                    append_next_edge(edges, e_i, mesh, vi, v2)
                
                #edges with counterclockwise higher index than ref edge
                edges = []
                append_next_edge(edges, central_edges[k], mesh, vi, v2)
                for e_i in edges:            
                    #calculate edge length
                    e_len = edge_length(e_i, mesh)
                    coeff = (1)/(e_len*Nc) 
                    row_ind[j] = parent
                    col_ind[j] = dual_space.GetDofNrs(e_i)[0]
                    data[j] = coeff * s
                    j += 1

                    append_next_edge(edges, e_i, mesh, vi, v2)
    #BND Edges ------------------------------------------------------------
    else:
        #deal with the one central edge
        e_len = edge_length(central_edges[0], mesh)
        coeff = (1)/(2*e_len) 
        row_ind[j] = parent
        col_ind[j] = dual_space.GetDofNrs(central_edges[0])[0]
        data[j] = coeff * -1
        j += 1

        for k in range(2):
            vi = verts[k]       
            s = sign[k]     
            Nc = len(vi.faces)/2
        
            #deal with reference edge (part of original edge)
            ref_edge = get_ref_edge(central_edges[0], mesh, vi, v2)
            e_len = edge_length(ref_edge, mesh)
            coeff = (2 - Nc)/(2*e_len*Nc) 
            row_ind[j] = parent
            col_ind[j] = dual_space.GetDofNrs(ref_edge)[0]
            data[j] = coeff * s
            j += 1

            numerator = [(1-Nc), (1)]
            edges = []
            append_next_edge(edges, central_edges[0], mesh, vi, v2)
            for e_i in edges:            
                #calculate edge length
                e_len = edge_length(e_i, mesh)
                coeff = numerator[k]/(e_len*Nc) 
                row_ind[j] = parent
                col_ind[j] = dual_space.GetDofNrs(e_i)[0]
                data[j] = coeff * s 
                j += 1

                append_next_edge(edges, e_i, mesh, vi, v2)

dual_map = la.SparseMatrixdouble.CreateFromCOO(row_ind,col_ind,data, nedges,mesh.nedge - nedges)
#print(dual_map.shape)

In [74]:
dfu = GridFunction(dual_space,multidim = unref_dofnr)
#help(ng_mat)

dfu.Set((0,0))
for i in range(unref_dofnr):
    #print(ng_mat.ToDense().NumPy()[:,i])
    dfu.vecs[:][i] = dual_map.ToDense().NumPy()[i,:]

# for i in range(4):
    #print(dfu.vecs[i][:].FV().NumPy())

Draw(dfu, animate = False, min = 0, max = 30, deformation = False)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene